# 😻 Find a Pet and Take it Home 

## Background story:
One eposide of the Korean TV show, Animal Farm, a stray kitten's mother died, the kitten did know what death meant and continued to be around and brought food she found to her dead mother while eating tree leaves herself. I was extremely touched by the story. 

This story inspired us to find stray animals homes!

After talking to my friend working as volunteer in a shelter, we think pictures matters! 
**Better pictures increase the chances for pets find homes!**

**Purpose** : We want to find what features make a better picture

### Project Outline: 
1. EDA Summary
 - Basic EDA for overall features
 - Pictures demonstration
 - EDA for individual features
2. PCA + Cluster Analysis
3. Decision Tree
4. CNN image Model

### Team Member
@Eileen_ds @Kathleen Lee @XiannvLiu

In [ ]:
from IPython.display import Image
Image("../input/kitty1/k.jpg")

<img src="https://media.giphy.com/media/vFKqnCdLPNOKc/giphy.gif" width="100" height="100" />

In [ ]:
# packages
from IPython.display import Image
from IPython.display import HTML, display
from PIL import Image, ImageFilter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.graphics.mosaicplot import mosaic
import warnings 
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import plot_tree
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler 
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

trainpt = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")


# 🖼  EDA Exploratory Analysis
we will go over the distribution of overall scores and the association between individual features and overall scores.

In [ ]:
trainpt.head()

## Overall Distribution
we categorize the pictures to 4 levels, A,B,C,D .
If the picture is in the A category, it has a better chance for the animal to be adopted.

- A: pawpoularity >85, More likely to take the pet home
- B: 60-85, It is a sweet picture, will consider to take home
- C: 40-60, It is a cute picture
- D: <40. We should take a better picture for the animals

In [ ]:
# Cut Level for Powpularity
powcat=pd.cut(trainpt.Pawpularity, bins=[0,40,60,85,100], 
             labels=["D", "C" ,"B", "A"])
trainpt.insert(14,"Powcat", powcat)

### Bar plot and Violin Plot of Pawpularity Categories

As we can see, the bar plot shows maority pictures are in group D, which means we should improve our picture taking skills .

The violin plot indicates most 40, 60, and 100 are common scores.
It is a little queer that there's a lot of 100, whereas few 80-95 records.

In [ ]:
sns.set()
sns.countplot(trainpt["Powcat"],palette="Set2")

In [ ]:
fig=plt.figure()
ax=sns.violinplot(x="Powcat" , y="Pawpularity",palette="Set2", data=trainpt)
plt.show()

### Overall Score Distribution

The overall score distribution shows the consistent info from previous violin and bar plot, most of pictures score around 30-40 interval,and 100. 

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
fig=plt.figure()
sns.histplot(data=trainpt, x="Pawpularity",kde=True, bins=100)
plt.axvline(trainpt["Pawpularity"].mean(),c="pink",ls="-", lw=3, label="Mean")
plt.axvline(trainpt["Pawpularity"].median(),c="darkseagreen",ls="-",lw=3,label="Median")
plt.title("Distribution of Pawpularity Scores", fontsize=20,fontweight="bold")
plt.legend()
plt.show()

### Heatmap between features

We can see there is no strong association of Pawpularity with any given features.
but we do notice 3 interesting correlations: 
1. Face and Eyes
2. Collage and Info
3. Human and Occlusion

In [ ]:
pet_ft=trainpt.drop(["Id","Pawpularity","Powcat"], axis=1)
y=trainpt.Pawpularity
plt.figure(figsize=(20,10))
sns.heatmap(trainpt.corr(),annot=True)
plt.show()

### Mosaic Plot for strong correlated variables


In [ ]:
ctable1= pd.crosstab(pet_ft["Face"],pet_ft["Eyes"])
ctable1
mosaic(data=pet_ft, index=["Face","Eyes"], title="Face vs Eyes")
plt.show()
mosaic(data=pet_ft, index=["Collage","Info"], title="Collage vs Info")
plt.show()
mosaic(data=pet_ft, index=["Human","Occlusion"], title="Human vs Occlusion")
plt.show()

## Pictures demonstration


In [ ]:
path = '../input/petfinder-pawpularity-score/'
train = pd.read_csv(path + 'train.csv')
test= pd.read_csv(path + 'test.csv')
train_jpg = glob(path + "train/*.jpg")
test_jpg = glob(path + "test/*.jpg")

In [ ]:
id_modified=train_jpg[0].split('/')[-1].split('.')[0]
train_df = train.loc[train['Id'] == id_modified]
train_df

In [ ]:
id_modified1 = train.loc[train['Id'] == id_modified,'Pawpularity']
pawpularity= id_modified1.iloc[0]
pawpularity

In [ ]:
for i in range(800,808):
    id_modified=train_jpg[i].split('/')[-1].split('.')[0]
    id_modified_series = train.loc[train['Id'] == id_modified,'Pawpularity']
    pawpularity= id_modified_series.iloc[0]
    image= plt.imread(train_jpg[i]) 
    plt.imshow(image)
    title =train_jpg[i] + '   -   ' + str(pawpularity)
    plt.title(title) 
    plt.show()

In [ ]:
def image_2_c_pawpularity(desired_pawpularity):
    showing_image_numbers=5
    sample=train[train["Pawpularity"] == desired_pawpularity].sample(5).reset_index(drop=True)
    for i in range(5):
        id_modified = sample.iloc[i]['Id']
        root = '../input/petfinder-pawpularity-score/train/'
        extension = '.jpg'
        image_path = root + str(id_modified) + extension
        pawpularity=sample.iloc[i]['Pawpularity']
        image= plt.imread(image_path)
        plt.subplot(1, 5, i+1)
        title = pawpularity
        plt.title(title) 
        plt.imshow(image)
    plt.show()

In [ ]:
image_2_c_pawpularity(20)

In [ ]:
image_2_c_pawpularity(40)

In [ ]:
image_2_c_pawpularity(60)

In [ ]:
image_2_c_pawpularity(80)

In [ ]:
image_2_c_pawpularity(100)

## Individual Feature EDA

This section we will look each features idvidually 

In [ ]:
def fplot(var,label):
    fig, ax=plt.subplots(1,3)
    sns.violinplot(x=var, y="Pawpularity",data=trainpt,palette="Set2",ax=ax[0])
    sns.histplot(trainpt,x="Pawpularity", hue=var, palette="Set2",kde=True,ax=ax[1])
    sns.countplot(x=var,hue="Powcat",data=trainpt,palette="Set2", ax=ax[2])
    plt.suptitle(label)
    return plt
fig.show()  

In [ ]:
fplot("Eyes", "Eye")

In [ ]:
fplot("Face", "Picture with Face")

In [ ]:
fplot("Near", "Pictures with Close position")

In [ ]:
fplot("Action", "Pets Picture with action")

In [ ]:
fplot("Accessory", "Pet's Picture with Accessory")

In [ ]:
fplot("Info", "Picture with info")

In [ ]:
fplot("Group", "Pet's Picture with Group")

In [ ]:
fplot("Collage", "Pet's Picture with Collage")

In [ ]:
fplot("Human", "Pet's Picture with Human")

In [ ]:
fplot("Occlusion", "Pet's Picture with Accessory")

In [ ]:
fplot("Blur", "Pet's Picture Blur")

# 🧮 Modeling 
## PCA to Select important features
We use PCA to select the most important features

In [ ]:
pca=PCA(n_components=5)
pca.fit(pet_ft)
print(pca.explained_variance_ratio_)
x_pca=pca.fit_transform(pet_ft)
ps=pd.DataFrame(x_pca)
ps.head()

In [ ]:
#try
topc=pd.DataFrame(ps[[3,4]])
print(topc.shape)
print(topc.head())
fig=plt.figure(figsize=(8,8))
plt.plot(topc[3],topc[4],"o")
plt.xlabel("Features")
plt.ylabel("y")
plt.legend()
plt.title("PCA Cluster ")
plt.show()

In [ ]:
#Choose the best cluster number
cloud=KMeans(n_clusters=4,random_state=8).fit(topc)
centers=cloud.cluster_centers_
c_pred=cloud.predict(topc)

In [ ]:
fig=plt.figure(figsize=(8,8))
colors=["palegreen", "orange","orchid","cyan"]
colored=[colors[k] for k in c_pred]
plt.scatter(topc[3],topc[4], color=colored)
for ci,c in enumerate(centers):
    plt.plot(c[0],c[1],"o",color="red", alpha=0.9)
plt.xlabel("x")
plt.ylabel("y")
plt.title("PCA Cluster Analysis")
plt.show()

In [ ]:
pet_ft=pet_ft.copy()
pet_ft["cluster"]=c_pred
pet_ft["label"]=trainpt.Powcat
pet_ft.head(10)
pd.crosstab(index=pet_ft["cluster"],columns=pet_ft["label"])

## 🌴 Decision Tree

We choose features instead all 12 features to be more reader friendly, the accuracy result is 0.66, we think it acceptable, but than flip a coin. ^_^ 

In [ ]:
## Decision Tree
# we tried use all features, the picture is too big to see the details,
# so we will use 4 feature for the purpose of demonstration.

target1=pet_ft["label"]
pet1t=pet_ft[["Blur","Accessory","Group"]]
#define attributes
X1=pet1t
target1

In [ ]:
#label encoding
le1=LabelEncoder()
target1=le1.fit_transform(target1)
y1=target1

#splitting data - 80:20
Xtrain1, Xtest1, ytrain1, ytest1= train_test_split (X1,y1,test_size=0.2,random_state=80)
print("Training split input-", Xtrain1.shape)
print("Testing split input-", Xtest1.shape)

In [ ]:
#prediction
dtree1=DecisionTreeClassifier()
dtree1.fit(Xtrain1,ytrain1)
#predicting test
ypred1=dtree1.predict(Xtest1)
print("Classification report -\n", classification_report(ytest1,ypred1))

In [ ]:
#tree plot
plt.figure(figsize=(50,50))
d_tree1=plot_tree(decision_tree=dtree1,feature_names=pet1t.columns,
                class_names=["A","B","C","D"],
                filled=True, precision=2)

## 🕸 CNN


**Convolutional Neural Network**

1.**Purpose**:  Extract Features from the picture
\
Patch match: common features
\
Filter Idea: divide block of pictures as feature (3x3) or (5x5)
\
$$\int_{-\infty}^{\infty}f{(\gamma)g(x-\gamma)d\gamma}$$


2.**Convolution: Feature Map**
A *filter* is responsible for detecting whether a feature exists in the data or not.
\
1 represent exact match
\
-1 represent exact opposite match
\
0 represent uncorrelated
\
*Stride* the interval how the filter moves
\
the blue window is the initial window, we can slide it.
\
stride=1, move one step to the right


3.**ReLU**
remove the uncorrelated parts
$$ f(x)= max (0,x) $$

4. **Pooling**: Max Pooling, Average Pooling

In [ ]:
def train_id_to_path(x):
    return '/kaggle/input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '/kaggle/input/petfinder-pawpularity-score/test/' + x + ".jpg"
traint = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
train11=traint[["Id","Pawpularity"]]
testt = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test11 = testt[["Id"]] 
train11["img_path"] = train11["Id"].apply(train_id_to_path)
test11["img_path"] = test11["Id"].apply(test_id_to_path)

In [ ]:
#standardize all image
image_height = 128
image_width = 128
def path_to_eagertensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
X1 = []
for img in train11['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X1.append(new_img_tensor)
    
print(type(X1),len(X1))
X1 = np.array(X1)
print(type(X1),X1.shape)

In [ ]:
X1_sub = []
for img in test11['img_path']:
    new_img_tensor = path_to_eagertensor(img)
    X1_sub.append(new_img_tensor)
    
print(type(X1_sub),len(X1_sub))
X1_sub = np.array(X1_sub)
print(type(X1_sub),X1_sub.shape)
y1=train11["Pawpularity"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
inputs = tf.keras.Input(shape=(image_height,image_width,3))
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2),padding='valid',activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same',activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='mse', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"]) 

In [ ]:
data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

In [ ]:
history = model.fit(
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    epochs = 3
)

In [ ]:
def convert2onehot_4(labels):
    result = []
    for label in labels:
#         print(label)
        if label == '0':
#             print("label 1")
            result.append([1, 0, 0, 0])
        elif label == '1':
            result.append([0,1,0,0])
        elif label == '2':
            result.append([0,0,1,0])
        else:
#             print("label 0")
            result.append([0,0,0,1])
    return np.array(result)
x_train_array = np.array(x_train)
x_test_array = np.array(x_test)
y_train_onehot = convert2onehot_4(y_train)
y_test_onehot = convert2onehot_4(y_test)

In [ ]:
inputs1 = tf.keras.Input(shape=(image_height,image_width,3))
x1 = inputs1

x1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2),padding='valid',activation = 'relu')(x1)
x1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)


x1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Dropout(0.25)(x1)

x1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x1)
x1 = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same',activation = 'relu')(x1)
x1 = tf.keras.layers.BatchNormalization()(x1)
x1 = tf.keras.layers.Dropout(0.25)(x1)

x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.layers.Dense(512, activation = "relu")(x1)
x1 = tf.keras.layers.Dropout(0.5)(x1)

output1 = tf.keras.layers.Dense(4)(x1)

model1 = tf.keras.Model(inputs = inputs1, outputs = output1)

In [ ]:
model1.compile(optimizer='Adam', loss='mse', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"]) 

In [ ]:
print(x_train_array.shape, y_train_onehot.shape,x_test_array.shape, y_test_onehot.shape)

In [ ]:
model1.summary()

In [ ]:
model1.fit(x_train_array, y_train_onehot, batch_size=16, epochs=3, validation_data=(x_test_array, y_test_onehot))

In [ ]:
cnn_pred = model.predict(X1_sub)
print(X1_sub.shape, type(X1_sub))
print(cnn_pred.shape, type(cnn_pred))

In [ ]:
cnn = pd.DataFrame()
cnn['Id'] = test11['Id']
cnn['Pawpularity'] = cnn_pred
cnn.to_csv('submission.csv',index=False)

In [ ]:
cnn.head(10)

In [ ]:
cnn_pred1 = model1.predict(X1_sub)
print(X1_sub.shape, type(X1_sub))
print(cnn_pred1.shape, type(cnn_pred1))

In [ ]:
cnn1=pd.DataFrame(cnn_pred1)
cnn1

# 📝 Summary

It seems like we didn't find a great way to classify what is a good picture through EDA, Decision Tree. Luckily @Kathleen_Lee spend a long time to tune the CNN, and we achieved a relative good result.
\
Besides the current data, maybe shelter can provide more info about the pets including:
- Spay, Nude status
- Age
- Pet Sex
- Vaccine status
- Charteristics of the pet

Those info are very valuable to the people who want to adopt animals.


This project perhaps won't help many pets to find homes, however we think what really counts is the warm intention and little friendly actions in our daily lives. Maybe we can't take stray animals home, but at least we could give them a bowl of soup or a sweet hola to them ! ♥️

### Thanks for reading! 

# 📚 Reference

1. Decision tree
https://www.analyticsvidhya.com/blog/2020/10/all-about-decision-tree-from-scratch-with-python-implementation/
2. Pet EDA
https://www.kaggle.com/alexteboul/tutorial-part-1-eda-for-beginners
3. CNN
https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1
https://www.kaggle.com/kathleenlee/petfinder-pawpularity
https://www.kaggle.com/eileends/pet-cnn
4. Picture info
https://www.kaggle.com/kathleenlee/petfinder-pawpularity
5. CNN Knowledge
https://zhuanlan.zhihu.com/p/27908027?utm_source=wechat_session&utm_medium=social&utm_oi=910092190892576768


## Pet Friendly 
1. 和猫住 With Cat 
https://space.bilibili.com/26321770/

2. TV animal farm https://en.wikipedia.org/wiki/Animals_(South_Korean_TV_series)

https://k.sina.com.cn/article_3120226247_b9fadfc70270112mc.html?sudaref=www.google.com&display=0&retcode=0

3. NJ NY cat adoption
   Wechat: jiansu1030